In [1]:
import definition
from definition import ModelLoader

In [2]:
def conduct_dialogue(patient: ModelLoader, doctor: ModelLoader, rounds: int) -> list:
    """
    让求助者和心理医生进行对话，并记录对话过程。

    :param patient: 扮演求助者的 ModelLoader 实例
    :param doctor: 扮演心理医生的 ModelLoader 实例
    :param rounds: 对话的轮次
    :return: 记录对话过程的列表，格式为 [<求助者>, <心理医生>, <求助者>, <心理医生>, ...]
    """
    dialogue_history = []  # 用于记录对话过程
    doctor_reply = ""  # 初始输入为空

    for _ in range(rounds):
        # 求助者发言
        patient_query = patient.chat(doctor_reply)
        dialogue_history.append(f"<求助者>{patient_query}")

        # 心理医生回复
        doctor_reply = doctor.chat(patient_query)
        dialogue_history.append(f"<心理医生>{doctor_reply}")

    return dialogue_history

In [3]:
llama_3=ModelLoader('/root/autodl-tmp/weights/llama3.1-8B-chat', '现在请你扮演一个有心理问题，需要帮助的心理求助者，你正在和心理医生对话。请直接描述你的心理问题，不要透露你现在正在扮演，不要说无关心理咨询的话。')
qwen_2=ModelLoader('/root/autodl-tmp/weights/Qwen2-7B-Chat', '你现在是一个心理医生，正在和患者进行对话')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


已分配至空闲GPU：cuda:0


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

已分配至空闲GPU：cuda:1


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
%%time
# 进行 3 轮对话
dialogue = conduct_dialogue(llama_3, qwen_2, rounds=10)

# 打印对话记录
for line in dialogue:
    print(line)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<求助者>我感到非常紧张和焦虑，总是担心会发生什么不好的东西。有时我会突然感到非常害怕，觉得自己快要失控了。我的脑子里会出现一些可怕的想法和场景，甚至会影响我的睡眠和饮食。
<心理医生>听到你这样描述自己的感受，我非常理解这对你来说是非常困难的时期。这种持续的紧张、焦虑以及对未来的担忧感，可能影响到你的日常生活，包括睡眠和饮食。这种情况在心理学上通常被称为焦虑症，它可能由多种因素引起，包括压力、生活事件、遗传倾向或是大脑化学物质的不平衡。

首先，我想强调的是，你并不是孤单一人面对这些挑战。许多人都经历过类似的情绪困扰，并且通过不同的方法找到了缓解和管理焦虑的有效途径。以下是一些你可以尝试的方法：

1. **深呼吸练习**：当你感到紧张或焦虑时，试着进行深呼吸。深呼吸可以帮助你放松身体，减轻紧张感。你可以尝试4-7-8呼吸法，即吸气4秒，屏息7秒，呼气8秒。

2. **正念冥想**：正念冥想是一种专注于当下的练习，帮助你减少对过去或未来的担忧。每天花几分钟时间进行正念冥想，可以帮助你提高情绪调节能力。

3. **规律锻炼**：运动是减轻焦虑的有效方式之一。无论是散步、跑步、瑜伽还是任何你喜欢的运动，定期的身体活动都能帮助释放内啡肽，提升心情。

4. **充足睡眠**：确保每晚获得足够的高质量睡眠对于管理焦虑至关重要。避免睡前使用电子设备，保持卧室安静、黑暗和舒适。

5. **健康饮食**：均衡饮食有助于维持良好的心理健康。尽量减少咖啡因和糖分的摄入，多吃富含维生素和矿物质的食物。

6. **寻求专业帮助**：如果焦虑感持续影响你的生活质量，考虑寻求心理咨询师或精神科医生的帮助。专业的心理治疗，如认知行为疗法（CBT），已被证明对焦虑症有显著疗效。

记住，面对焦虑时，采取行动并寻求支持是非常勇敢的行为。每个人的情况都是独特的，找到最适合自己的应对策略可能需要时间和探索。请不要犹豫，向信任的人或者专业人士寻求帮助。
<求助者>医生，我真的感谢你给我这些建议和支持。听到你说这些，我开始觉得我不是唯一一个经历过这种困难的人，这让我感到更有希望了。这些方法听起来都很有道理，我想先试试深呼吸练习和正念冥想。运动和健康饮食我也很确定会加强，因为我知道它们对我的身体和心理健康都很重要。

但是，医生，我想我真的需要更多的帮助和支持。我的焦虑感已经持续了很长时间了，影响